In [2]:
# testing collab feature - marija
import ee
ee.Initialize(project='gist-itproject2')  

#NEW MESSAGE 
#CAN YOU SEE THIS
#omg yes this is so cool 
#better than word! also does running the code work 

In [5]:
# --- deps ---
import ee, json, os, zipfile, geopandas as gpd, pandas as pd
import geemap                     # for ee_to_geojson()
import geemap.foliumap as gf      # Folium-backed Map
import folium
import pandas as pd
import geopandas as gpd

ee.Initialize(project='gist-itproject2')

# -------------------------------
# 1) Indo-Pacific list (LSIB names)
# -------------------------------
ip_names = [
    "Afghanistan","Bangladesh","Bhutan","India","Maldives","Nepal","Pakistan","Sri Lanka",
    "Brunei","Burma","Cambodia","Indonesia","Laos","Malaysia","Philippines","Singapore",
    "Thailand","Timor-Leste","Vietnam",
    "China","Hong Kong","Macau","Japan","Korea, South","Korea, North","Mongolia","Taiwan",
    "Australia","Fiji","Kiribati","Marshall Islands","Micronesia, Federated States of","Nauru",
    "New Zealand","Palau","Papua New Guinea","Samoa","Solomon Islands","Tonga","Tuvalu","Vanuatu"
]

profiles = {
    "Australia":{"Capital":"Canberra","Ruling":"Australian Labor Party",
                 "Opposition":"Liberal-Nationals Coalition",
                 "Religion1":"No Religion (38.7%)","Religion2":"Catholic (19.6%)"},
    "China":{"Capital":"Beijing","Ruling":"Chinese Communist Party (CCP)",
             "Opposition":"None (one-party state)",
             "Religion1":"Unaffiliated (52.1%)","Religion2":"Chinese Folk Religion (21.9%)"},
    "India":{"Capital":"New Delhi","Ruling":"BJP-led coalition","Opposition":"INDIA bloc",
             "Religion1":"Hinduism (79.8%)","Religion2":"Islam (14.2%)"},
    "Indonesia":{"Capital":"Nusantara (transitional)","Ruling":"Gerindra (President)",
                 "Opposition":"PDI-P (role in flux)",
                 "Religion1":"Muslim (87.2%)","Religion2":"Protestant (7%)"},
    "Japan":{"Capital":"Tokyo","Ruling":"LDP–Komeito coalition","Opposition":"CDP",
             "Religion1":"Shinto (48.6%)","Religion2":"Buddhism (46.4%)"},
    "Malaysia":{"Capital":"Kuala Lumpur","Ruling":"Pakatan Harapan (PH)",
                "Opposition":"Perikatan Nasional (PN)",
                "Religion1":"Islam (63.5%)","Religion2":"Buddhism (18.7%)"},
    "New Zealand":{"Capital":"Wellington","Ruling":"National-led coalition","Opposition":"Labour",
                   "Religion1":"No Religion (51.6%)","Religion2":"Christianity (32.3%)"},
    "Philippines":{"Capital":"Manila","Ruling":"Alyansa para sa Bagong Pilipinas",
                   "Opposition":"DuterTen coalition",
                   "Religion1":"Roman Catholic (78.8%)","Religion2":"Islam (6.4%)"},
    "Korea, South":{"Capital":"Seoul","Ruling":"People Power Party (PPP)",
                    "Opposition":"Democratic Party (DP)",
                    "Religion1":"Unaffiliated (46.5%)","Religion2":"Christian (29%)"},
    "Thailand":{"Capital":"Bangkok","Ruling":"Pheu Thai coalition",
                "Opposition":"People's Party (formerly MFP)",
                "Religion1":"Theravada Buddhism (90–95%)","Religion2":"Muslim (~5%)"},
    "Vietnam":{"Capital":"Hanoi","Ruling":"Communist Party of Vietnam (CPV)",
               "Opposition":"None (one-party)",
               "Religion1":"Irreligion/Folk Religion (76.5%)","Religion2":"Buddhism (13.3%)"}
}

# -------------------------------
# 2) EE data & attributes
# -------------------------------
world = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
ip_fc = world.filter(ee.Filter.inList('country_na', ip_names))
non_ip_fc = world.filter(ee.Filter.Not(ee.Filter.inList('country_na', ip_names)))

def set_profile(f):
    name = ee.String(f.get('country_na'))
    prof = ee.Dictionary(profiles).get(name, ee.Dictionary({}))
    prof = ee.Dictionary(prof)
    return f.set({
        'Country': name,
        'Capital': prof.get('Capital', ''),
        'RulingParty': prof.get('Ruling', ''),
        'Opposition': prof.get('Opposition', ''),
        'Religion1': prof.get('Religion1', ''),
        'Religion2': prof.get('Religion2', '')
    })

ip_fc = ip_fc.map(set_profile)

# diamond centroids (one per country)
centroids = (
    ip_fc
    .map(lambda f: ee.Feature(f.geometry().centroid(1), f.toDictionary()))
    .distinct('Country')
)

# -------------------------------
# 3) Folium map
# -------------------------------
m = gf.Map(center=[-10, 140], zoom=3)
m.add_basemap("CartoDB.Positron")

# EE styled overlays
non_ip_style = non_ip_fc.style(color="d0d0d0", fillColor="5ca2ff", width=0.2)
ip_fill      = ip_fc.style(color="ffffff", fillColor="ffcc00", width=1.1)
ip_outline   = ip_fc.style(color="ffffff", fillColor="00000000", width=2.0)

m.addLayer(non_ip_style, {}, "Outside Indo-Pacific", True, 0.18)
m.addLayer(ip_fill, {}, "Indo-Pacific (fill)", True, 0.95)
m.addLayer(ip_outline, {}, "Indo-Pacific (outline)", True, 1.0)

# -------------------------------
# 4) Country diamond markers
# -------------------------------
gj = geemap.ee_to_geojson(centroids)

diamond_icon_html = """
<div style="width:14px;height:14px;background:#ffcc00;border:2px solid white;
            transform: rotate(45deg);box-shadow:0 0 3px rgba(0,0,0,.35);
            border-radius:1px;"></div>"""

marker_group = folium.FeatureGroup(name="Country markers", show=True)

for feat in gj["features"]:
    lon, lat = feat["geometry"]["coordinates"]
    p = feat["properties"]

    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">{p.get('Country','')}</div>
      <div><b>Capital:</b> {p.get('Capital','') or '—'}</div>
      <div><b>Ruling:</b> {p.get('RulingParty','') or '—'}</div>
      <div><b>Opposition:</b> {p.get('Opposition','') or '—'}</div>
      <div><b>Religions:</b> {p.get('Religion1','') or '—'}{', ' if p.get('Religion2') else ''}{p.get('Religion2','')}</div>
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        icon=folium.DivIcon(html=diamond_icon_html)
    ).add_to(marker_group)

marker_group.add_to(m)

# -------------------------------
# 5) Volcanoes (Red Triangles) from CSV
# -------------------------------

#CHANGE PATH HERE TO YOUR LOCAL PATH
volcano_df = pd.read_csv("/Users/kurtischen/Downloads/Mainmap/datasets/GVP_Volcano_List_Holocene_202504090341_edit.csv")

# Filter only rows with valid coordinates
volcano_df = volcano_df[volcano_df['Latitude'].notnull() & volcano_df['Longitude'].notnull()]


volcano_layer = folium.FeatureGroup(name="Volcanoes", show=False)

# Volcano type to color mapping
type_colors = {
    "Stratovolcano": "red",
    "Shield volcano": "orange",
    "Caldera": "purple",
    "Lava dome": "blue",
    "Complex volcano": "green",
    "Pyroclastic cone": "pink",
    "Submarine volcano": "cyan",
    "Maar": "brown"
}

# Add volcano markers
for _, row in volcano_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    vtype = row.get('Primary Volcano Type', '')
    color = type_colors.get(vtype, 'gray')  # fallback if type missing

    triangle_icon_html = f"""
    <div style="
      width: 0;
      height: 0;
      border-left: 7px solid transparent;
      border-right: 7px solid transparent;
      border-bottom: 14px solid {color};
      transform: rotate(0deg);
    "></div>
    """

    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">{row.get('Volcano Name','')}</div>
      <div><b>Country:</b> {row.get('Country','—')}</div>
      <div><b>Last Eruption:</b> {row.get('Last Known Eruption','—')}</div>
      <div><b>Elevation:</b> {row.get('Elevation (m)','—')} m</div>
      <div><b>Type:</b> {vtype or '—'}</div>
      <div><b>Rock Type:</b> {row.get('Dominant Rock Type','—')}</div>
      <div><b>Tectonic Setting:</b> {row.get('Tectonic Setting','—')}</div>
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.DivIcon(html=triangle_icon_html)
    ).add_to(volcano_layer)

volcano_layer.add_to(m)

# -------------------------------
# 6) Tectonic Plates
# -------------------------------
plate_color_map = {
    'spreading center': 'blue',
    'collision zone': 'red',
    'subduction zone': 'darkred',
    'transform': 'orange',
    'extension zone': 'purple',
    'inferred': 'gray'
}
# Load simplified Hasterok plate boundaries
#CHANGE PATH HERE TO YOUR LOCAL PATH
plates = gpd.read_file("/Users/kurtischen/Downloads/Mainmap/datasets/TectonicPlates/Hasterok_plate_boundaries_simplified.shp")
plates = plates.to_crs(epsg=4326)  # Make sure it's WGS84 for Folium

# Ensure correct CRS (EPSG:4326)
if plates.crs and plates.crs.to_epsg() != 4326:
    plates = plates.to_crs(epsg=4326)

# Add to map as styled GeoJSON
plate_layer = folium.FeatureGroup(name="Tectonic Plates", show=True)

for boundary_type, color in plate_color_map.items():
    subset = plates[plates['type'].str.lower().str.strip() == boundary_type]

    if not subset.empty:
        folium.GeoJson(
            subset,
            name=boundary_type.title(),  # "Spreading Center"
            style_function=lambda feature, color=color: {
                "color": color,
                "weight": 2.5,
                "opacity": 0.9
            },
            tooltip=folium.GeoJsonTooltip(fields=['type', 'plate1', 'plate2'])
        ).add_to(plate_layer)

plate_layer.add_to(m)

# -------------------------------
# 8) Cyclones (Tropical Cyclones) from IBTrACS dataset
# -------------------------------
# Define shapefile path (relative)
cyclone_files = "datasets/Cyclones/IBTrACS.ALL.list.v04r01.lines.shp"

# Check if shapefile exists
if not os.path.exists(cyclone_files):
    cyclone_dir = "datasets/Cyclones"
    print(f"Shapefile not found at {cyclone_files}. Available files in directory:")
    print(os.listdir(cyclone_dir))
    raise FileNotFoundError(f"Cannot find {cyclone_files}")

# Load IBTrACS shapefile (lines) for cyclones
try:
    cyclones_gdf = gpd.read_file(cyclone_files)
    print("Shapefile loaded successfully. First few rows:")
    print(cyclones_gdf.head())
except Exception as e:
    print(f"Error loading shapefile: {e}")
    raise

# Filter to Indo-Pacific region (approximate bounding box: lon 40 to 180, lat -50 to 50)
cyclones_ip = cyclones_gdf.cx[40:180, -50:50]

# Filter to recent data (2000-2024) to focus on modern events
cyclones_ip = cyclones_ip[cyclones_ip['SEASON'] >= 2000]

# Filter to tropical storms and above (USA_WIND >= 34 knots)
cyclones_ip = cyclones_ip[cyclones_ip['USA_WIND'] >= 34]

# Sample to limit data for testing (e.g., 100 cyclones)
cyclones_ip = cyclones_ip.sample(n=100, random_state=42)  # random_state for reproducibility

# Define wind speed categories for intensity shading (Saffir-Simpson inspired)
def get_wind_category(wind):
    if wind < 64:
        return 'Tropical Storm'
    elif wind < 83:
        return 'Category 1'
    elif wind < 96:
        return 'Category 2'
    elif wind < 113:
        return 'Category 3'
    elif wind < 137:
        return 'Category 4'
    else:
        return 'Category 5'

wind_colors = {
    'Tropical Storm': '#ADD8E6',  # Light blue
    'Category 1': '#0000FF',     # Blue
    'Category 2': '#00008B',     # Dark blue
    'Category 3': '#800080',     # Purple
    'Category 4': '#FFA500',     # Orange
    'Category 5': '#FF0000'      # Red
}

# Create cyclone layer (points extracted from line centroids, colored by intensity)
cyclone_layer = folium.FeatureGroup(name="Cyclones", show=False)

for _, row in cyclones_ip.iterrows():
    # Get centroid of the line geometry for marker placement
    lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
    wind = row['USA_WIND']
    category = get_wind_category(wind)
    color = wind_colors.get(category, 'gray')
    
    # Circle radius proportional to wind speed (scale: 3-20 pixels)
    radius = max(3, min(20, (wind / 10)))
    
    # Icon HTML for a filled circle
    circle_icon_html = f"""
    <div style="
        width: {2*radius}px;
        height: {2*radius}px;
        background-color: {color};
        border: 2px solid #333;
        border-radius: 50%;
        opacity: 0.8;
        box-shadow: 0 0 5px rgba(0,0,0,0.5);
    "></div>
    """
    
    # Popup with cyclone details
    name = row.get('NAME', 'Unknown')
    iso_time = row.get('ISO_TIME', 'Unknown')
    basin = row.get('BASIN', 'Unknown')
    pressure = row.get('USA_PRES', 'N/A')
    
    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">{name} ({category})</div>
      <div><b>Wind Speed:</b> {wind} knots</div>
      <div><b>Date/Time:</b> {iso_time}</div>
      <div><b>Basin:</b> {basin}</div>
      <div><b>Pressure:</b> {pressure} mb</div>
    </div>
    """
    
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        icon=folium.DivIcon(html=circle_icon_html)
    ).add_to(cyclone_layer)

cyclone_layer.add_to(m)

# Cyclone legend (toggleable)
cyclone_legend_html = '''
<div id="cyclone-legend" class="legend" style="position: fixed; bottom: 10px; right: 10px; width: 150px; height: 100px; 
     background-color: white; border: 2px solid grey; z-index: 9999; font-size: 12px; padding: 5px; display: none;">
  <h4 style="margin: 0 0 4px 0; font-size: 14px;">Cyclone</h4>
  <i style="background:#ADD8E6; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Tropical Storm<br>
  <i style="background:#0000FF; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Category 1<br>
  <i style="background:#00008B; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Category 2<br>
  <i style="background:#800080; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Category 3<br>
  <i style="background:#FFA500; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Category 4<br>
  <i style="background:#FF0000; width:12px; height:12px; display:inline-block; margin-right:5px;"></i> Category 5
</div>
'''
m.get_root().html.add_child(folium.Element(cyclone_legend_html))


# -------------------------------
# 9) Earthquakes from ISC-GEM dataset
# -------------------------------
# Define earthquake CSV path (relative)
earthquake_file = "datasets/Earthquakes/isc-gem-cat-edit.csv"

# Check if file exists
if not os.path.exists(earthquake_file):
    print(f"CSV not found at {earthquake_file}")
    raise FileNotFoundError(f"Cannot find {earthquake_file}")

# Load earthquake CSV without header (first row is data, not headers)
earthquake_df = pd.read_csv(earthquake_file, header=None)

# Assign column names based on the structure you provided earlier
column_names = [
    'date', 'lat', 'lon', 'smajax', 'sminax', 'strike', 'q', 'depth', 'unc', 'q .1',
    'mw', 'unc', 'q .2', 's', 'mo', 'fac', 'mo_auth', 'mpp', 'mpr', 'mrr',
    'mrt', 'mtp', 'mtt', 'str1', 'dip1', 'rake1', 'str2', 'dip2', 'rake2',
    'type', 'eventid'
]
earthquake_df.columns = column_names

print("Available columns in earthquake CSV:", earthquake_df.columns.tolist())
print("Sample dates from 'date' column:", earthquake_df['date'].head().tolist())

# Define column names
lon_col = 'lon'         # Longitude column (index 2)
lat_col = 'lat'         # Latitude column (index 1)
mag_col = 'mw'          # Magnitude column (index 10)
date_col = 'date'       # Date column (index 0)
depth_col = 'depth'     # Depth column (index 7)
region_col = 'type'     # Using 'type' as a proxy for region (index 29)

# Convert numeric columns to float, coercing errors to NaN
numeric_cols = [lon_col, lat_col, mag_col, depth_col]
for col in numeric_cols:
    earthquake_df[col] = pd.to_numeric(earthquake_df[col], errors='coerce')

# Parse the date column with the correct format (YYYY-MM-DD HH:MM:SS.sss) and keep as datetime64[ns]
try:
    earthquake_df[date_col] = pd.to_datetime(earthquake_df[date_col], format='%Y-%m-%d %H:%M:%S.%f')
except ValueError as e:
    print(f"Date parsing error: {e}. Trying flexible parsing...")
    earthquake_df[date_col] = pd.to_datetime(earthquake_df[date_col], errors='coerce')

# Drop rows where any numeric column or date is NaN (unparseable)
earthquake_df = earthquake_df.dropna(subset=numeric_cols + [date_col])

# Filter to Indo-Pacific region (lon: 40 to 180, lat: -50 to 50)
earthquake_df = earthquake_df[
    (earthquake_df[lon_col] >= 40) & (earthquake_df[lon_col] <= 180) &
    (earthquake_df[lat_col] >= -50) & (earthquake_df[lat_col] <= 50)
]

# Filter to recent data (2000-2025) and magnitude >= 4.0 using datetime64[ns]
earthquake_df = earthquake_df[
    (earthquake_df[date_col] >= pd.to_datetime('2000-01-01')) & 
    (earthquake_df[date_col] <= pd.to_datetime('2025-09-13')) &  # Updated to current date
    (earthquake_df[mag_col] >= 4.0)
]

# Sample to limit data for testing (e.g., 100 earthquakes)
earthquake_df = earthquake_df.sample(n=100, random_state=42)  # random_state for reproducibility

# Print filtered data to debug
print("Filtered earthquake data sample:")
print(earthquake_df.head())

# Magnitude-based coloring: Highlight strong quakes (M7+) in red/orange
def get_mag_color(mag):
    if mag < 5:
        return '#00FF00'  # Green
    elif mag < 6:
        return '#FFFF00'  # Yellow
    elif mag < 7:
        return '#FFA500'  # Orange
    else:
        return '#FF0000'  # Red (strong, e.g., subduction-related)

# Create earthquake layer
# Create earthquake layer
earthquake_layer = folium.FeatureGroup(name="Earthquakes", show=False)

for _, row in earthquake_df.iterrows():
    lat = row[lat_col]
    lon = row[lon_col]
    mag = row[mag_col]
    color = get_mag_color(mag)
    
    # Radius proportional to magnitude (scale: 3-25 pixels)
    radius = max(3, min(25, mag * 3))
    
    # Icon HTML for circle
    circle_icon_html = f"""
    <div style="
        width: {2*radius}px;
        height: {2*radius}px;
        background-color: {color};
        border: 2px solid #333;
        border-radius: 50%;
        opacity: 0.8;
        box-shadow: 0 0 5px rgba(0,0,0,0.5);
    "></div>
    """
    
    # Popup with details
    time = row[date_col].strftime('%Y-%m-%d') if pd.notnull(row[date_col]) else 'Unknown'
    depth = row.get(depth_col, 'N/A')
    mag = row.get(mag_col, 'N/A')
    etype = row.get(region_col, 'Unknown')

    popup_html = f"""
    <div style="font-family:system-ui;min-width:240px;line-height:1.35">
      <div style="font-weight:700;margin-bottom:4px">Magnitude {mag}</div>
      <div><b>Lat/Lon:</b> {lat}, {lon}</div>
      <div><b>Date:</b> {time}</div>
      <div><b>Depth:</b> {depth} km</div>
      <div><b>Type:</b> {etype}</div>
    </div>
    """

    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=280),
        icon=folium.DivIcon(html=circle_icon_html)
    ).add_to(earthquake_layer)

earthquake_layer.add_to(m)

# -------------------------------
# Master Legend (all layers)
# -------------------------------
master_legend_html = """
<div id="map-legend" style="
    position: fixed;
    bottom: 20px;
    left: 20px;
    width: 240px;
    max-height: 400px;
    overflow-y: auto;
    background-color: white;
    border: 2px solid grey;
    border-radius: 8px;
    z-index: 9999;
    font-size: 12px;
    padding: 10px;
    box-shadow: 0 0 8px rgba(0,0,0,0.3);
    line-height: 1.4;
">
  <h4 style="margin-top:0; font-size:14px; text-align:center;">Legend</h4>

  <!-- Indo-Pacific -->
  <b>Indo-Pacific Region</b><br>
  <span style="display:inline-block;width:18px;height:12px;background:#ffcc00;
         border:1px solid #999;margin-right:6px;"></span> Indo-Pacific country<br>
  <span style="display:inline-block;width:18px;height:12px;background:#5ca2ff;
         border:1px solid #999;margin-right:6px;"></span> Other country<br>
  <div style="width:14px;height:14px;background:#ffcc00;border:2px solid white;
              transform:rotate(45deg);display:inline-block;margin-right:6px;"></div> Country centroid<br><br>

  <!-- Volcanoes -->
  <b>Volcanoes</b><br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid red;
              display:inline-block;margin-right:6px;"></div> Stratovolcano<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid orange;
              display:inline-block;margin-right:6px;"></div> Shield volcano<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid purple;
              display:inline-block;margin-right:6px;"></div> Caldera<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid blue;
              display:inline-block;margin-right:6px;"></div> Lava dome<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid green;
              display:inline-block;margin-right:6px;"></div> Complex volcano<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid pink;
              display:inline-block;margin-right:6px;"></div> Pyroclastic cone<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid cyan;
              display:inline-block;margin-right:6px;"></div> Submarine volcano<br>
  <div style="width:0;height:0;border-left:7px solid transparent;
              border-right:7px solid transparent;border-bottom:14px solid brown;
              display:inline-block;margin-right:6px;"></div> Maar<br><br>

  <!-- Tectonic Plates -->
  <b>Tectonic Plate Boundaries</b><br>
  <span style="background:blue;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Spreading center<br>
  <span style="background:red;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Collision zone<br>
  <span style="background:darkred;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Subduction zone<br>
  <span style="background:orange;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Transform<br>
  <span style="background:purple;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Extension zone<br>
  <span style="background:gray;width:20px;height:3px;display:inline-block;margin-right:6px;"></span> Inferred<br><br>

  <!-- Cyclones -->
  <b>Cyclones</b><br>
  <i style="background:#ADD8E6; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Tropical Storm<br>
  <i style="background:#0000FF; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Category 1<br>
  <i style="background:#00008B; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Category 2<br>
  <i style="background:#800080; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Category 3<br>
  <i style="background:#FFA500; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Category 4<br>
  <i style="background:#FF0000; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Category 5<br><br>

  <!-- Earthquakes -->
  <b>Earthquakes</b><br>
  <i style="background:#00FF00; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Magnitude 4–4.9<br>
  <i style="background:#FFFF00; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Magnitude 5–5.9<br>
  <i style="background:#FFA500; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Magnitude 6–6.9<br>
  <i style="background:#FF0000; width:12px; height:12px; display:inline-block; margin-right:6px;"></i> Magnitude ≥7<br>
</div>
"""
m.get_root().html.add_child(folium.Element(master_legend_html))

# -------------------------------
# 7) Final controls and display
# -------------------------------
folium.LayerControl(collapsed=False).add_to(m)

# Show in notebook
m



Shapefile loaded successfully. First few rows:
             SID  SEASON  NUMBER BASIN SUBBASIN     NAME             ISO_TIME  \
0  1842298N11080    1842       1    NI       BB  UNNAMED  1842-10-25 03:00:00   
1  1842298N11080    1842       1    NI       BB  UNNAMED  1842-10-25 06:00:00   
2  1842298N11080    1842       1    NI       BB  UNNAMED  1842-10-25 09:00:00   
3  1842298N11080    1842       1    NI       BB  UNNAMED  1842-10-25 12:00:00   
4  1842298N11080    1842       1    NI       BB  UNNAMED  1842-10-25 15:00:00   

  NATURE   LAT   LON  ...  USA_SEA_SW  USA_SEA_NW STORM_SPD STORM_DR  year  \
0     NR  10.9  80.3  ...         NaN         NaN         9      265  1842   
1     NR  10.9  79.8  ...         NaN         NaN         9      265  1842   
2     NR  10.8  79.4  ...         NaN         NaN         9      265  1842   
3     NR  10.8  78.9  ...         NaN         NaN         9      265  1842   
4     NR  10.8  78.4  ...         NaN         NaN         9      270  1842  

/var/folders/q5/p4y1y3q55hn186drps2c4jmh0000gn/T/ipykernel_40061/3883049297.py:373: DtypeWarning: Columns (1,2,3,4,5,7,8,10,11,30) have mixed types. Specify dtype option on import or set low_memory=False.
  earthquake_df = pd.read_csv(earthquake_file, header=None)
/var/folders/q5/p4y1y3q55hn186drps2c4jmh0000gn/T/ipykernel_40061/3883049297.py:405: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  earthquake_df[date_col] = pd.to_datetime(earthquake_df[date_col], errors='coerce')


Available columns in earthquake CSV: ['date', 'lat', 'lon', 'smajax', 'sminax', 'strike', 'q', 'depth', 'unc', 'q .1', 'mw', 'unc', 'q .2', 's', 'mo', 'fac', 'mo_auth', 'mpp', 'mpr', 'mrr', 'mrt', 'mtp', 'mtt', 'str1', 'dip1', 'rake1', 'str2', 'dip2', 'rake2', 'type', 'eventid']
Sample dates from 'date' column: ['date          ', ' 1904-04-04 10:02:34.56 ', ' 1904-04-04 10:26:00.88 ', ' 1904-06-25 14:45:39.14 ', ' 1904-06-25 21:00:38.72 ']
Date parsing error: time data "date          " doesn't match format "%Y-%m-%d %H:%M:%S.%f", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.. Trying flexible parsing...
Filtered earthquake data sample:
                         date     lat      

In [2]:
m.to_html("MainMap.html")